In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip -q install configparser langchain openai
!pip -q install transformers huggingface_hub 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.4 MB/s eta 0:00:00


In [ ]:
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.llms import OpenAi

In [ ]:
#Then convert the transformers pipeline into fact_llm 

fact_llm = OpenAI(temperature=0.1)

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt_open = PromptTemplate(template=template, input_variables=["question"])

open_chain = LLMChain(prompt=prompt_open,llm = fact_llm)

In [ ]:
question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

print(open_chain.run(question))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


 Justin Beiber was born in 1994, so the Super Bowl was in 2015. The team that won the Super Bowl in 2015 was the New England Patriots.


In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
recurSplitter = RecursiveCharacterTextSplitter(chunk_size=100,
                                               chunk_overlap=20,
                                               length_function=len)

In [ ]:
with open('/content/linux_play.txt') as lin:
  txt_lin = lin.read()

linux_docs = recurSplitter.create_documents([txt_lin])

In [ ]:
with open('/content/space_shortened.csv') as spc:
  txt_spc = spc.read()

space_docs = recurSplitter.create_documents([txt_spc])

In [ ]:
from langchain.embeddings import OpenAIEmbeddings()
hfEmbed = OpenAIEmbeddings()

In [ ]:
from langchain.vectorstores import Chroma
space_directory = 'space_db'

space_chroma = Chroma.from_documents(documents=space_docs,
                                     embeddings=hfEmbed,
                                     persist_directory=space_directory)

In [ ]:
lin_directory = 'lin_db'

lin_chroma = Chroma.from_documents(documents=linux_docs,
                                     embeddings=hfEmbed,
                                     persist_directory=lin_directory)

In [ ]:
# Starting the custom tool route

In [ ]:
lin_retriever =RetrievalQA.from_chain_type(llm=fact_llm, 
                                           chain_type="stuff", 
                                           retriever=lin_chroma.as_retriever())

In [ ]:
space_retriever =RetrievalQA.from_chain_type(llm=fact_llm, 
                                           chain_type="stuff", 
                                           retriever=space_chroma.as_retriever())

In [ ]:
tools = [
    Tool(
        name = "Space Farers",
        func=space_retriever.run,
        description="useful for when you need to answer questions about the space farers."
    ),
    Tool(
        name = "Linux Commands",
        func=lin_retriever.run,
        description="useful for when you need to answer questions about Linux commands."
    ),
]

In [ ]:
example_agent = initialize_agent(tools, 
                         fact_llm, 
                         agent="zero-shot-react-description", 
                         verbose=True)

In [ ]:
example_agent.save("Custom_tool_VS.json")

In [ ]:
example_agent.run("How many space farers are there?")

In [ ]:
# Starting the custom vectorstore agent route

In [ ]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_router_agent,
    VectorStoreRouterToolkit,
    VectorStoreInfo,
)

In [ ]:
space_vectorstore_info = VectorStoreInfo(
    name="space",
    description="Used when need to answer about Space fares",
    vectorstore=space_chroma
)

In [ ]:
linux_vectorstore_info = VectorStoreInfo(
    name="linux",
    description="Used when need to answer about playbook",
    vectorstore=lin_chroma
)

In [ ]:
router_toolkit = VectorStoreRouterToolkit(
    vectorstores=[space_vectorstore_info,
                  linux_vectorstore_info],
    llm=fact_llm
)
agent_executor = create_vectorstore_router_agent(
    llm=fact_llm,
    toolkit=router_toolkit,
    verbose=True
)

In [ ]:
query = """How many Space fares are there?"""

In [ ]:
agent_executor.run(query)

In [ ]:
agent_executor.save_agent("multi_doc_agent.json")